In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import os
import imageio as io
from scipy.interpolate import griddata
from cartopy.io.img_tiles import OSM
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
df = pd.read_csv('data/data_tot.csv')

In [ ]:
# Convertim la columna de data en un objecte datetime
df['DATA_LECTURA'] = pd.to_datetime(df['DATA_LECTURA'])

# Creem una nova columna amb només la data (sense l'hora)
df['date'] = df['DATA_LECTURA'].dt.date
df['year']= df['DATA_LECTURA'].dt.year

# Calculem la mitjana, màxima i mínima diàries
daily_mean = df.groupby(['date', 'year', 'NOM_ESTACIO', 'LATITUD', 'LONGITUD'])['Temperatura'].mean().reset_index(name='Temperatura mitjana')
daily_max = df.groupby(['date', 'year', 'NOM_ESTACIO', 'LATITUD', 'LONGITUD'])['Temperatura'].max().reset_index(name='Temperatura màxima')
daily_min = df.groupby(['date', 'year', 'NOM_ESTACIO', 'LATITUD', 'LONGITUD'])['Temperatura'].min().reset_index(name='Temperatura mínima')

In [ ]:
# Fusionem els DataFrames un a un
data_temp = pd.merge(daily_mean, daily_max, on=['date', 'year', 'NOM_ESTACIO', 'LATITUD', 'LONGITUD'])
data_temp = pd.merge(data_temp, daily_min, on=['date', 'year', 'NOM_ESTACIO', 'LATITUD', 'LONGITUD'])

In [ ]:
# Comprovem que la carpeta on volem guardar les imatges existeix. En cas contrari, la crearem
path='imatges'
if os.path.exists(path)==False:
    os.makedirs(path)

In [ ]:
# Generem el cmap per al heatmap de temperatures
cmap = plt.get_cmap('turbo', 256)

In [ ]:
for d in data_temp['date'].values:
  dades = data_temp[data_temp['date'] == d]

  for col in dades.columns[5:]:
    temp_min = dades[col].min()
    temp_max = dades[col].max()
    vmin = temp_min
    vmax = temp_max

    tiler = OSM()
    fig, ax = plt.subplots(subplot_kw={'projection': tiler.crs})
    ax.set_extent([dades['LONGITUD'].min()-0.005, dades['LONGITUD'].max()+0.005,
                   dades['LATITUD'].min()-0.005, dades['LATITUD'].max()+0.005],
                  crs=ccrs.PlateCarree())

    ax.add_image(tiler, 12)

    lon = np.linspace(dades['LONGITUD'].min(), dades['LONGITUD'].max(), 100)
    lat = np.linspace(dades['LATITUD'].min(), dades['LATITUD'].max(), 100)
    lon_grid, lat_grid = np.meshgrid(lon, lat)

    points = dades[['LONGITUD', 'LATITUD']].values
    values = dades[col].values
    grid_z = griddata(points, values, (lon_grid, lat_grid), method='linear')

    pcm = ax.pcolormesh(lon_grid, lat_grid, grid_z, cmap=cmap, shading='auto', alpha=0.7, transform=ccrs.PlateCarree(), vmin=vmin, vmax=vmax)
    cbar = plt.colorbar(pcm, ax=ax, orientation='vertical', shrink=0.5, extend='both')
    cbar.set_label(f'{col} (°C)')

    # Título del mapa
    ax.set_title(f'{col} en Barcelona: {d}')

    # Mostrar la figura
    plt.savefig(f'{path}/{col} {d}.png')
    plt.close(fig)

In [ ]:
# Obtenim tots els noms de totes les imatges generades
imatges = os.listdir(path)
temps = ['Temperatura mitjana', 'Temperatura màxima', 'Temperatura mínima']

# Per cada tipus de temperatura afagem les imatges generades i creem els GIFs
for t in temps:
    im_gif = []
    for i in imatges:
        if i.startswith(t):
            im_gif.append(i)
    with io.get_writer(f'{t}.gif', mode='I', duration=1000) as writer:
        for filename in im_gif:
            pth = os.path.join(path, filename)
            image = io.imread(pth)
            writer.append_data(image)
    writer.close()
